[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/alexfleury/cph409/blob/main/notebooks/Chimie_quantique.ipynb)

# Résolution numérique de l'équation de Schrödinger pour des molécules

Nous avons vu dans le cours différents scénarios où l'équation de Schrödinger a été résolu pour obtenu les fonctions et valeurs propres.

$$
\hat{H} \Psi = E \Psi
$$

Comme il est coutume en chimie quantique, nous utiliserons l'approximation de Born–Oppenheimer ($\Psi = \psi_N \psi_e$) et considèrerons seulement la partie électronique du système. L'Hamiltonien associé aux électrons est donnée par

$$
\hat{H}_e = -\frac{\hbar^2}{2 m_e} \sum_i \nabla_i^2 -\frac{e^2}{4 \pi \epsilon_0} \sum_i \sum_A \frac{Z_A}{r_{iA}} + \frac{e^2}{4 \pi \epsilon_0} \sum_i \sum_{j>i} \frac{1}{r_{ij}}
$$

où le premier, deuxième et troisième termes font respectivement référence aux énergies cinétiques des électrons, l'attraction électron-noyau et la répulsion électron-électron. Pour de petits systèmes à un électron ($\text{H}_2^+$, $\text{HeH}^{2+}$, ...), il est possible d'obtenir les solutions analytiques des fonctions propres et des valeurs propres de l'équation de Schrödinger. La fonction d'onde peut alors être connue pour ces petits systèmes. Pour des systèmes compliqués (deux électrons ou plus!), une approximation de leur fonction d'onde peut être calculé numériquement à l'aide d'approximations. La formulation de ces approximations définit la méthode de calcul. Dans les prochains exemples, nous utiliserons
- Méthode Hartree-Fock (HF)
- Théorie de la fonctionnelle de la densité (DFT)

Ces méthodes sont implémentées dans plusieurs codes et logiciels tels que [Gaussian](https://gaussian.com/), [ORCA](https://orcaforum.kofo.mpg.de/app.php/portal), [Quantum Espresso](https://www.quantum-espresso.org/), [VASP](https://www.vasp.at/) et bien d'autres. Dans ce notebook, nous utiliserons [PySCF](https://pyscf.org/), qui a l'avantage d'être gratuit, open-source et facile d'utilisation.

## Installation de PySCF

La première étape est d'installer les fonctions nécessaires à nos calculs. L'utilisation de PySCF s'avère utile pour traiter des molécules au niveau de la chimie quantique. La cellule suivante installera PySCF si ce dernier n'est pas déjà installé dans votre environement.

In [5]:
try:
    import pyscf
except ImportError:
    !pip install pyscf

## Importation des fonctions

Ce ne sont pas toutes les fonctions qui seront pertinentes pour ce travail pratique. Du module `pyscf`, nous importons
* `gto`: Pour définir une molécule sur un ensemble de base gaussien (*Gaussian Type Orbitals*);
* `scf`: *Self-consistent field* pour résoudre l'équation de Schrödinger avec la méthode Hartree-Fock;
* `tools`: Divers outils, nous l'utiliserons pour créer des fichiers `.cube` pour visualiser des orbitales moléculaires.

In [6]:
from pyscf import gto, scf, tools

## Définition de la molécule

L'étape suivante est de définir la molécule. La variable `xyz` contient les coordonnées (x, y et z) de chaque atome. Nous créons ensuite une molécule avec la fonction `gto.Mole`, qui accepte les coordonnées des atomes, la charge totale, le spin (différence entre le nombre d'électrons alpha et beta) et l'ensemble de base. La commande `mol.build` construit les quantités nécessaires pour les calculs subséquents.

La cellule suivante contruit un objet python pour la molécule du dihydrogène neutre, singulet et avec un ensemble de base minimal de type Slater STO-3G.

In [16]:
xyz = """
H  0.000000    0.000000    0.000000
F  0.000000    0.000000    0.950000
"""

mol = gto.Mole(
    atom = xyz,
    charge = 0,
    spin = 0,
    basis = "STO-3G",
    verbose=4
)
mol.build()

System: uname_result(system='Linux', node='archCornet', release='5.17.9-arch1-1', version='#1 SMP PREEMPT Wed, 18 May 2022 17:30:11 +0000', machine='x86_64', processor='')  Threads 4
Python 3.8.5 (default, Mar  2 2022, 17:20:03) 
[GCC 11.2.0]
numpy 1.22.3  scipy 1.8.0
Date: Sat Jun  4 23:20:21 2022
PySCF version 2.0.1
PySCF path  /home/alexandre/VirtualEnvs/quantum/lib/python3.8/site-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 2
[INPUT] num. electrons = 10
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT]  1 H      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.000000000000 Bohr
[INPUT]  2 F      0.000000000000   0.000000000000   0.950000000000 AA    0.000000000000   0.000000000000   1.795239818337 Bohr

nuclear repulsion = 5.01325778766316
number of shells = 4
number of NR pGTOs = 18
number of NR cGTOs = 6
basis = STO-3G

Notons que nous utilisons ici un ensemble de base dit *minimal* ou de simple zeta. Ceci veut dire que l'ensemble de bases ne représentent seulement que le minimum des orbitals définis pour un élément. Dans le cas de HF,

| Atome | Orbital(s)        | Nombre |
|-------|-------------------|--------|
| 1 F   | 1s+2s+2px+2py+2pz | 5      |
| 1 H   | 1s                | 1      |

Six orbitals atomiques sont alors mélangés pour donner six orbitals moléculaires.

## Calcul en Hartree-Fock

L'appel au module `scf.RHF` rend accessible la fonction d'onde approximée avec la méthode Hartree-Fock, ou plus précisément la solution *restricted Hartree-Fock*. Le mot *restricted* fait référence à la méthode qui force les électrons à être pairés (2 électrons par orbitale moléculaire). D'autres méthodes peuvent être qualifiées de `unrestricted`, où cette contrainte n'est pas imposée. C'est à la ligne `hf.kernel()` où le calcul se produit et que le résultat est affiché.

In [17]:
hf = scf.RHF(mol)
hf.kernel()



******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /home/alexandre/Projects/cph409/notebooks/tmp1c_9dh5y
max_memory 4000 MB (current use 122 MB)
Set gradient conv threshold to 3.16228e-05
init E= -98.3186515639205
  HOMO = -0.566940085888614  LUMO = 0.377561408852904
cycle= 1 E= -98.5236354509471  delta_E= -0.205  |g|= 0.315  |ddm|= 1.18
  HOMO = -0.345520960332502  LUMO = 0.590838773864142
cycle= 2 E= -98.572029671899  delta_E= -0.0484  |g|= 0.0401  |ddm|= 0.52
  HOMO = -0.461483134006855  LUMO = 0.595586952345562
cycle= 3 E= -98.5727990557326  delta_E= -0.000769  |g|= 0.00532  |ddm|= 0.0767
  HOMO = -0.463153018821442  LUMO = 0.594947514278995
cycle= 4 E= -98.5728082639284  delta_E= -9.21e-06  |g|= 0.00022

-98.57280828758289

Le résultat de `-98.572808` est l'énergie de la molécule en hartree (1 hartree vaut 627.5 kcal/mol). L'appel à la fonction `analyze` imprime diverses quantités utiles:
* Les orbitals moléculaires (énergie et nombre d'électrons);
* Les charges sur chaque atome;
* Le moment dipolaire de la molécule.

In [18]:
hf.analyze();

**** SCF Summaries ****
Total Energy =                         -98.572808287582887
Nuclear Repulsion Energy =               5.013257787663158
One-electron Energy =                 -149.439799566636339
Two-electron Energy =                   45.853733491390287
**** MO energy ****
MO #1   energy= -25.9030060865052  occ= 2
MO #2   energy= -1.46136620537621  occ= 2
MO #3   energy= -0.575307623203942 occ= 2
MO #4   energy= -0.463243158957823 occ= 2
MO #5   energy= -0.463243158957823 occ= 2
MO #6   energy= 0.59526538771883   occ= 0
 ** Mulliken pop on meta-lowdin orthogonal AOs  **
 ** Mulliken pop  **
pop of  0 H 1s        0.85422
pop of  1 F 1s        1.99996
pop of  1 F 2s        1.88455
pop of  1 F 2px       2.00000
pop of  1 F 2py       2.00000
pop of  1 F 2pz       1.26126
 ** Mulliken atomic charges  **
charge of  0H =      0.14578
charge of  1F =     -0.14578
Dipole moment(X, Y, Z, Debye):  0.00000,  0.00000, -1.25709


## Calcul avec la théorie de la fonctionelle de la densité

In [19]:
dft = mol.KS()
dft.xc = "b3lyp"
dft.kernel()



******** <class 'pyscf.dft.rks.RKS'> ********
method = RKS-RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /home/alexandre/Projects/cph409/notebooks/tmpa_nui_eg
max_memory 4000 MB (current use 129 MB)
XC library pyscf.dft.libxc version 5.1.7
    S. Lehtola, C. Steigemann, M. J. Oliveira, and M. A. Marques, SoftwareX 7, 1 (2018)
XC functionals = b3lyp
    P. A. M. Dirac, Math. Proc. Cambridge Philos. Soc. 26, 376 (1930)
    F. Bloch, Z. Phys. 57, 545 (1929)
    A. D. Becke, Phys. Rev. A 38, 3098 (1988)
    C. Lee, W. Yang, and R. G. Parr, Phys. Rev. B 37, 785 (1988)
    B. Miehlich, A. Savin, H. Stoll, and H. Preuss, Chem. Phys. Lett. 157, 200 (1989)
    S. H. Vosko, L. Wilk, and M. Nusair, Can. J. Phys. 58, 1200 (1980)
radial grids: 
    Treutler-Ahlrichs [J

-98.8861313150308

In [20]:
dft.analyze();

**** SCF Summaries ****
Total Energy =                         -98.886131315030795
Nuclear Repulsion Energy =               5.013257787663158
One-electron Energy =                 -149.389853302079871
Two-electron Coulomb Energy =           56.427115996779165
DFT Exchange-Correlation Energy =      -10.936651797393253
**** MO energy ****
MO #1   energy= -24.3028100078037  occ= 2
MO #2   energy= -1.04882742439006  occ= 2
MO #3   energy= -0.359683725651575 occ= 2
MO #4   energy= -0.171967105074106 occ= 2
MO #5   energy= -0.171967105074105 occ= 2
MO #6   energy= 0.345512497130182  occ= 0
 ** Mulliken pop on meta-lowdin orthogonal AOs  **
 ** Mulliken pop  **
pop of  0 H 1s        0.86454
pop of  1 F 1s        1.99995
pop of  1 F 2s        1.89188
pop of  1 F 2px       2.00000
pop of  1 F 2py       2.00000
pop of  1 F 2pz       1.24362
 ** Mulliken atomic charges  **
charge of  0H =      0.13546
charge of  1F =     -0.13546
Dipole moment(X, Y, Z, Debye):  0.00000,  0.00000, -1.16678


## Analyze/visualisation des orbitales moléculaires

blabla orbital HF sont un peu mieux que DFT etc.

In [21]:
tools.dump_mat.dump_mo(mol, hf.mo_coeff)

               #0        #1        #2        #3        #4       
  0 H 1s      -0.00540   0.15157  -0.53203   0.00000   0.00000
  1 F 1s       0.99474  -0.25047  -0.07886   0.00000   0.00000
  1 F 2s       0.02231   0.94528   0.41426  -0.00000   0.00000
  1 F 2px      0.00000   0.00000   0.00000   1.00000   0.00000
  1 F 2py     -0.00000   0.00000  -0.00000   0.00000   1.00000
  1 F 2pz     -0.00271  -0.07983   0.69860   0.00000   0.00000
               #5       
  0 H 1s       1.05921
  1 F 1s       0.08118
  1 F 2s      -0.52179
  1 F 2px     -0.00000
  1 F 2py      0.00000
  1 F 2pz      0.81758


In [22]:
for i in range(hf.mo_coeff.shape[1]):
    tools.cubegen.orbital(mol, f"orbitale_{i+1:02d}.cube", hf.mo_coeff[:,i])